In [1]:
from os import path
import os
import pandas as pd
import requests, zipfile, io, os
from zipfile import BadZipfile
from zipfile import ZipFile
import shutil
import glob
import numpy as np
pd.set_option('display.max_colwidth', -1)

/var/folders/mk/7yynft554x38v3t6f_65fkzc0000gn/T/ipykernel_97470/64196590.py:10: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [2]:
df = pd.read_csv('AllRegion_EPWLinks.csv', index_col=0) 
df

,Location Name,Coordinates,URL for files
0,Vernadsky.Research.Base-Argentine.Islands GBR ATA,"['-64.26700', '-65.25000', '9.0']",https://climate.onebuilding.org/WMO_Region_7_Antarctica/ATA_Antarctica/ATA_GBR_Vernadsky.Research.Base-Argentine.Islands.889520_TMYx.zip
1,Esperanza.Base-Hope.Bay ARG ATA,"['-56.99730', '-63.39810', '3.0']",https://climate.onebuilding.org/WMO_Region_7_Antarctica/ATA_Antarctica/ATA_ARG_Esperanza.Base-Hope.Bay.889630_TMYx.2004-2018.zip
2,Esperanza.Base-Hope.Bay ARG ATA,"['-56.99730', '-63.39810', '3.0']",https://climate.onebuilding.org/WMO_Region_7_Antarctica/ATA_Antarctica/ATA_ARG_Esperanza.Base-Hope.Bay.889630_TMYx.2007-2021.zip
3,Esperanza.Base-Hope.Bay ARG ATA,"['-56.99730', '-63.39810', '3.0']",https://climate.onebuilding.org/WMO_Region_7_Antarctica/ATA_Antarctica/ATA_ARG_Esperanza.Base-Hope.Bay.889630_TMYx.zip
4,Orcadas.Base-Laurie.Island-South.Orkney.Islands ARG ATA,"['-44.73300', '-60.73300', '8.0']",https://climate.onebuilding.org/WMO_Region_7_Antarctica/ATA_Antarctica/ATA_ARG_Orcadas.Base-Laurie.Island-South.Orkney.Islands.889680_TMYx.2004-2018.zip
...,...,...,...
45935,Old Crow RCS YT CAN,"['-139.8400', '67.57000', '251.2']",https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/CAN_Canada/YT_Yukon/CAN_YT_Old.Crow.RCS.710440_CWEC2020.zip
45936,Rock River YT CAN,"['-136.2200', '66.98000', '731.0']",https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/CAN_Canada/YT_Yukon/CAN_YT_Rock.River.715060_CWEC2020.zip
45937,Teslin AUT YT CAN,"['-132.7300', '60.17000', '705.0']",https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/CAN_Canada/YT_Yukon/CAN_YT_Teslin.AUT.710450_CWEC2020.zip
45938,Watson Lake AP YT CAN,"['-128.8200', '60.12000', '687.3']",https://climate.onebuilding.org/WMO_Region_4_North_and_Central_America/CAN_Canada/YT_Yukon/CAN_YT_Watson.Lake.AP.711990_CWEC2020.zip


In [3]:
df['zip'] = df['URL for files'].str.split('/').str[-1]
df['zip']

0        ATA_GBR_Vernadsky.Research.Base-Argentine.Islands.889520_TMYx.zip                
1        ATA_ARG_Esperanza.Base-Hope.Bay.889630_TMYx.2004-2018.zip                        
2        ATA_ARG_Esperanza.Base-Hope.Bay.889630_TMYx.2007-2021.zip                        
3        ATA_ARG_Esperanza.Base-Hope.Bay.889630_TMYx.zip                                  
4        ATA_ARG_Orcadas.Base-Laurie.Island-South.Orkney.Islands.889680_TMYx.2004-2018.zip
                                               ...                                        
45935    CAN_YT_Old.Crow.RCS.710440_CWEC2020.zip                                          
45936    CAN_YT_Rock.River.715060_CWEC2020.zip                                            
45937    CAN_YT_Teslin.AUT.710450_CWEC2020.zip                                            
45938    CAN_YT_Watson.Lake.AP.711990_CWEC2020.zip                                        
45939    CAN_YT_Whitehorse.AP.719640_CWEC2020.zip                                         

In [13]:
df['WMO'] = df['zip'].str.extract('(\d+)')
df['WMO'] = df['WMO'].fillna(0)
df['WMO'] =df['WMO'].astype(str).astype(int)

df['WMO'].dtypes

dtype('int64')

In [14]:
df['last_year'] = df['zip'].str.split('.').str[-2].str.split('-').str[-1]
df['last_year'] = df['last_year'].astype('string')
df['last_year'] = df['last_year'].str[-4:]

df['last_year'].head()

0    TMYx
1    2018
2    2021
3    TMYx
4    2018
Name: last_year, dtype: string

In [18]:
def download_file(url):
    headers=   {"Auth": "{abcd}", 
                "accept": "*/*",               
               "accept-encoding": "gzip;deflate;br" }
    response = requests.request("GET", url, headers = headers)
    filename = os.getcwd()+'/downloads_EPW/'
    #filename = os.getcwd()+'/test_epw/'
    try:
        with ZipFile(io.BytesIO(response.content)) as z:
            print("zipfile is OK")
            #z = zipfile.ZipFile(io.BytesIO(response.content))
            z_files = z.namelist()
            #print(z_files)
            extension = '.epw'
            epw = [file for file in z_files if os.path.splitext(file)[1] == extension]
            epwName= epw[0]
            print(epwName)
            z.extract(epw[0],filename)
            #print(epw[0])
            #print(filename)
            #z.extractall(filename)
    except BadZipfile:
        print('Download did not work')
        epwName = "N/A"
    return response.status_code, epwName



In [19]:
## Read in an EPW file to a Pandas dataframe
def readEPW(path):
    # Read the location
    df = pd.read_csv(path, nrows=1, header=None,encoding_errors='ignore') # Read only the first row
    latitude = df.iloc[0, 6]
    longitude = df.iloc[0, 7]
    timezone = df.iloc[0, 8]
    
    # Read the hourly data
    df = pd.read_csv(path, skiprows=range(8), header=None,encoding_errors='ignore') # Read the file, skip the 8 header rows
    # There are no column names in the file. We need to assign them.
    df.columns = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Data Source and Uncertainty Flags',
                  'Dry Bulb Temperature', 'Dew Point Temperature', 'Relative Humidity', 'Atmospheric Station Pressure',
                  'Extraterrestrial Horizontal Radiation', 'Extraterrestrial Direct Normal Radiation', 'Horizontal Infrared Radiation Intensity',
                  'Global Horizontal Radiation', 'Direct Normal Radiation', 'Diffuse Horizontal Radiation',
                  'Global Horizontal Illuminance', 'Direct Normal Illuminance', 'Diffuse Horizontal Illuminance', 'Zenith Luminance',
                  'Wind Direction', 'Wind Speed', 'Total Sky Cover', 'Opaque Sky Cover', 'Visibility', 'Ceiling Height',
                  'Present Weather Observation', 'Present Weather Codes', 'Precipitable Water', 'Aerosol Optical Depth',
                  'Snow Depth', 'Days Since Last Snowfall', 'Albedo', 'Liquid Precipitation Depth', 'Liquid Precipitation Quantity']
    return df

In [20]:
## Read in an EPW file to a Pandas dataframe
def readEPW_DesignDay(path):
    # Read the location
    df = pd.read_csv(path, nrows=1, header=None,encoding_errors='ignore') # Read only the first row
    latitude = df.iloc[0, 6]
    longitude = df.iloc[0, 7]
    timezone = df.iloc[0, 8]
    
    # Read the hourly data
    df = pd.read_csv(path, skiprows=range(8), header=None,encoding_errors='ignore') # Read the file, skip the 8 header rows
    # There are no column names in the file. We need to assign them.
    df.columns = ['Year', 'Month', 'Day', 'Hour', 'Minute', 'Data Source and Uncertainty Flags',
                  'Dry Bulb Temperature', 'Dew Point Temperature', 'Relative Humidity', 'Atmospheric Station Pressure',
                  'Extraterrestrial Horizontal Radiation', 'Extraterrestrial Direct Normal Radiation', 'Horizontal Infrared Radiation Intensity',
                  'Global Horizontal Radiation', 'Direct Normal Radiation', 'Diffuse Horizontal Radiation',
                  'Global Horizontal Illuminance', 'Direct Normal Illuminance', 'Diffuse Horizontal Illuminance', 'Zenith Luminance',
                  'Wind Direction', 'Wind Speed', 'Total Sky Cover', 'Opaque Sky Cover', 'Visibility', 'Ceiling Height',
                  'Present Weather Observation', 'Present Weather Codes', 'Precipitable Water', 'Aerosol Optical Depth',
                  'Snow Depth', 'Days Since Last Snowfall', 'Albedo', 'Liquid Precipitation Depth', 'Liquid Precipitation Quantity']
    
    htg_996_epw= df['Dry Bulb Temperature'].quantile(q=0.004)
    htg_990_epw= df['Dry Bulb Temperature'].quantile(q=0.010)
    clg_004_epw= df['Dry Bulb Temperature'].quantile(q=0.996)
    clg_010_epw= df['Dry Bulb Temperature'].quantile(q=0.99)
    return htg_996_epw,htg_990_epw,clg_004_epw,clg_010_epw

In [15]:
#import DD data
df_DD = pd.read_excel('Climate Data 2021_SI.xlsx', index_col=0) 
df_DD

,Country,WMO,WBAN,Latitude,Longitude,Heating DB 99.6%,Heating DB 99%,Cooling DB 0.4%,Cooling MCWB 0.4%,Cooling DB 1%,Cooling MCWB 1%,Cooling DB 2%,Cooling MCWB 2%,Evaporation WB 0.4%,Evaporation MCDB 0.4%,Evaporation WB 1%,Evaporation MCDB 1%,Evaporation WB 2%,Evaporation MCDB 2%
Subregion/Station Name,,,,,,,,,,,,,,,,,,,
AN MAYEN,SvalbardandJanMayen,10010,99999,70.940N,8.668W,-14.0,-11.8,9.6,8.0,8.9,7.7,8.4,7.4,8.6,9.2,8.1,8.7,7.7,8.2
ORNSUND,SvalbardandJanMayen,10030,99999,77.000N,15.500E,-22.1,-20.5,8.1,6.1,7.2,5.6,6.6,5.2,6.6,7.5,6.0,6.8,5.5,6.3
DGEOYA,Norway,10060,99999,78.251N,22.823E,-28.0,-25.7,9.2,7.3,8.1,6.5,7.1,5.7,7.5,8.9,6.6,7.9,5.8,7.0
Y-ALESUND,SvalbardandJanMayen,10070,99999,78.9228N,11.9331E,-25.2,-22.7,9.9,6.8,8.7,6.1,7.9,5.6,7.3,9.3,6.5,8.4,5.8,7.6
VALBARD,SvalbardandJanMayen,10080,99999,78.246N,15.466E,-28.5,-25.8,11.9,8.3,10.5,7.5,9.4,6.9,8.7,11.1,7.8,9.8,7.1,8.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"EST NEEBISH ISLAND, MI",USA,998203,99999,46.2847N,84.2098W,-25.6,-22.4,27.9,NaN,26.3,NaN,24.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"AKE WATEREE, SC",USA,998208,99999,34.335N,80.702W,-5.5,-3.5,36.8,NaN,35.4,NaN,34.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"UCK PIER, NC",USA,998209,99999,36.184N,75.746W,-4.2,-2.1,31.4,NaN,30.2,NaN,29.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_summary = df.merge(df_DD, how='inner', on='WMO')
df_summary

In [ ]:
s_htg_996_epw= []
s_htg_990_epw= []
s_clg_004_epw= []
s_clg_010_epw= []

counter =0
#for url in df_summary['URL for files']:
for index in df_summary.index:
    if index > counter:
        #print(index)
        print(counter)
        counter=counter+1
        url = df_summary.loc[index,'URL for files']
        print(url)
        status, epwpath = download_file(url)
        print(status, epwpath)
        if status ==  200:
            htg_996,htg_990,clg_004,clg_010 = readEPW_DesignDay(path.join('downloads_EPW/',epwpath))
            #print(htg_996,htg_990,clg_004,clg_010)
            s_htg_996_epw.append(htg_996)
            s_htg_990_epw.append(htg_990)
            s_clg_004_epw.append(clg_004)
            s_clg_010_epw.append(clg_010)
            os.remove(path.join('downloads_EPW/',epwpath))
        else:
            s_htg_996_epw.append("N/A")
            s_htg_990_epw.append("N/A")
            s_clg_004_epw.append("N/A")
            s_clg_010_epw.append("N/A")
        continue

In [ ]:
df_summary['Heating DB 99.6% - EPW'] = s_htg_996_epw
df_summary['Heating DB 99% - EPW'] = s_htg_990_epw
df_summary['Cooling DB 0.4% - EPW'] = s_clg_004_epw
df_summary['Cooling DB 1% - EPW'] = s_clg_010_epw
df_summary

In [ ]:
filename = 'AllRegion_Results.csv'
export_csv = df_merge.to_csv(filename, index = None, header=True) 